# Introduction

**In this kernel i will try to share my understanding and findings of cross lingual models.Feel free to correct me if I made any mistakes in this kernel.**



**Imports**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import gc
import os
import time
import math
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import date
from transformers import *
from sklearn.metrics import *
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F



import warnings
warnings.filterwarnings("ignore")

import os
import gc
import re
import folium
#import textstat
from scipy import stats
from colorama import Fore, Back, Style, init

import math
import numpy as np
import scipy as sp
import pandas as pd

import random
import networkx as nx
from pandas import Timestamp

from PIL import Image
from IPython.display import SVG
from keras.utils import model_to_dot

import requests
from IPython.display import HTML

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
import matplotlib.pyplot as plt

tqdm.pandas()

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import transformers
import tensorflow as tf

from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from tensorflow.keras.models import Model
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.optimizers import Adam
from tokenizers import BertWordPieceTokenizer
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding
from tensorflow.keras.layers import LSTM, GRU, Conv1D, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.constraints import *
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *

from sklearn import metrics
from sklearn.utils import shuffle
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer,\
                                            CountVectorizer,\
                                            HashingVectorizer

from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer  

import nltk
from textblob import TextBlob

from nltk.corpus import wordnet
from nltk.corpus import stopwords
#from googletrans import Translator
from nltk import WordNetLemmatizer
#from polyglot.detect import Detector
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer

stopword=set(STOPWORDS)

lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

np.random.seed(0)




# Setup TPU configuration

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
print(strategy.num_replicas_in_sync)
#GCS_DS_PATH = KaggleDatasets().get_gcs_path('jigsaw-multilingual-toxic-comment-classification')
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

# Part 2 : Implementation using TPU Multiprocessing

In [ ]:
import pandas as pd

In [ ]:
# train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
# train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
# train2.toxic = train2.toxic.round().astype(int)

# train3 = pd.read_csv('../input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-es-cleaned.csv')

# train4 = pd.read_csv('../input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-es-cleaned.csv')

valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
# test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
# sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

# toxic = len(train2[['comment_text', 'toxic']].query('toxic==1'))
# # Combine train1 with a subset of train2
# train = pd.concat([
#     train1[['comment_text', 'toxic']],
#     train2[['comment_text', 'toxic']].query('toxic==1'),
    
#     train3[['comment_text', 'toxic']].query('toxic==0'),
#     train3[['comment_text', 'toxic']].query('toxic==1'),
    
#     train4[['comment_text', 'toxic']].query('toxic==0'),
#     train4[['comment_text', 'toxic']].query('toxic==1'),
    
    
#     train2[['comment_text', 'toxic']].query('toxic==0').sample(n=(toxic+(toxic//3)), random_state=101)
# ])

# test_data = test
# train_data = train

maxlen = 192

In [ ]:
# valid.head()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.figure(figsize=(10, 5))
# plt.subplot(1, 2, 1)
# sns.countplot(train_data['toxic'])
# plt.title('Target on training data')

# plt.subplot(1, 2, 2)
# sns.countplot(valid['toxic'])
# plt.title('Target on validation data')

# plt.show()

In [ ]:
# df_valid_en = pd.read_csv('../input/val-en-df/validation_en.csv')
# df_valid_en

In [ ]:




# df_valid_en = df_valid_en.drop(columns=['id', 'comment_text','lang'])
# df_valid_en = df_valid_en.rename(columns={"comment_text_en": "comment_text"})
# columns_titles = ["comment_text","toxic"]
# df_valid_en=df_valid_en.reindex(columns=columns_titles)

# '''df2 = df_valid_en[df_valid_en.toxic == 1]
# df_valid_en = pd.concat([df_valid_en,df2])
# df_valid_en = pd.concat([df_valid_en,df2])  #doubling twice
# '''

# print(df_valid_en.head(3))

# train_data = pd.concat([train_data , df_valid_en], axis=0).reset_index(drop=True)
# train_data = train_data.sample(frac=1).reset_index(drop=True)

# print(len(train2[['comment_text', 'toxic']].query('toxic==1')))
# print(len(train_data))
# train_data.head()



**Clean the text (remove usernames and links)**

In [ ]:
# val = valid
# train = train_data

# def clean(text):
#     text = text.fillna("fillna").str.lower()
#     text = text.map(lambda x: re.sub('\\n',' ',str(x)))
#     text = text.map(lambda x: re.sub("\[\[User.*",'',str(x)))
#     text = text.map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
#     text = text.map(lambda x: re.sub("\(http://.*?\s\(http://.*\)",'',str(x)))
#     return text

# val["comment_text"] = clean(val["comment_text"])
# test_data["content"] = clean(test_data["content"])
# train["comment_text"] = clean(train["comment_text"])

# More Text Cleaning

**applying text cleaning techniques like clean_text,replace_typical_misspell,handle_contractions,fix_quote 
on train,test and validation set**

In [ ]:
# # https://www.kaggle.com/chenshengabc/from-quest-encoding-ensemble-a-little-bit-differen

# puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
#  '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
#  '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
#  '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
#  '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

# mispell_dict = {"aren't" : "are not",
# "can't" : "cannot",
# "couldn't" : "could not",
# "couldnt" : "could not",
# "didn't" : "did not",
# "doesn't" : "does not",
# "doesnt" : "does not",
# "don't" : "do not",
# "hadn't" : "had not",
# "hasn't" : "has not",
# "haven't" : "have not",
# "havent" : "have not",
# "he'd" : "he would",
# "he'll" : "he will",
# "he's" : "he is",
# "i'd" : "I would",
# "i'd" : "I had",
# "i'll" : "I will",
# "i'm" : "I am",
# "isn't" : "is not",
# "it's" : "it is",
# "it'll":"it will",
# "i've" : "I have",
# "let's" : "let us",
# "mightn't" : "might not",
# "mustn't" : "must not",
# "shan't" : "shall not",
# "she'd" : "she would",
# "she'll" : "she will",
# "she's" : "she is",
# "shouldn't" : "should not",
# "shouldnt" : "should not",
# "that's" : "that is",
# "thats" : "that is",
# "there's" : "there is",
# "theres" : "there is",
# "they'd" : "they would",
# "they'll" : "they will",
# "they're" : "they are",
# "theyre":  "they are",
# "they've" : "they have",
# "we'd" : "we would",
# "we're" : "we are",
# "weren't" : "were not",
# "we've" : "we have",
# "what'll" : "what will",
# "what're" : "what are",
# "what's" : "what is",
# "what've" : "what have",
# "where's" : "where is",
# "who'd" : "who would",
# "who'll" : "who will",
# "who're" : "who are",
# "who's" : "who is",
# "who've" : "who have",
# "won't" : "will not",
# "wouldn't" : "would not",
# "you'd" : "you would",
# "you'll" : "you will",
# "you're" : "you are",
# "you've" : "you have",
# "'re": " are",
# "wasn't": "was not",
# "we'll":" will",
# "didn't": "did not",
# "tryin'":"trying"}


# def clean_text(x):
#     x = str(x).replace("\n","")
#     for punct in puncts:
#         x = x.replace(punct, f' {punct} ')
#     return x


# def clean_numbers(x):
#     x = re.sub('[0-9]{5,}', '#####', x)
#     x = re.sub('[0-9]{4}', '####', x)
#     x = re.sub('[0-9]{3}', '###', x)
#     x = re.sub('[0-9]{2}', '##', x)
#     return x

In [ ]:
# from nltk.tokenize.treebank import TreebankWordTokenizer
# tokenizer = TreebankWordTokenizer()

# def handle_contractions(x):
#     x = tokenizer.tokenize(x)
#     return x

# def fix_quote(x):
#     x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
#     x = ' '.join(x)
#     return x

# def _get_mispell(mispell_dict):
#     mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
#     return mispell_dict, mispell_re


# def replace_typical_misspell(text):
#     mispellings, mispellings_re = _get_mispell(mispell_dict)

#     def replace(match):
#         return mispellings[match.group(0)]

#     return mispellings_re.sub(replace, text)


# def clean_data(df, columns: list):
#     for col in columns:
# #         df[col] = df[col].apply(lambda x: clean_numbers(x))
#         df[col] = df[col].apply(lambda x: clean_text(x.lower())) 
#         df[col] = df[col].apply(lambda x: replace_typical_misspell(x))
#         df[col] = df[col].apply(lambda x: handle_contractions(x))  
#         df[col] = df[col].apply(lambda x: fix_quote(x))   
    
#     return df



In [ ]:
# %%time
# input_columns = [
#     'comment_text'   
# ]

# '''applying text cleaning techniques like clean_text,replace_typical_misspell,handle_contractions,fix_quote 
# on train,test and validation set'''

# train = clean_data(train, input_columns ) 
# val = clean_data(val, input_columns )


In [ ]:
# %%time
# input_columns = [
#     'content'   
# ]
# test_data = clean_data(test_data, input_columns )

# del tokenizer

In [ ]:
# val.to_csv('/kaggle/working/valid_cleaned.csv')
# test_data.to_csv('/kaggle/working/test_cleaned.csv')
# train.to_csv('/kaggle/working/train_cleaned.csv')

In [ ]:
# val = pd.read_csv('/kaggle/working/valid_cleaned.csv')
# test_data = pd.read_csv('/kaggle/working/test_cleaned.csv')
# train = pd.read_csv('/kaggle/working/train_cleaned.csv')

**we can see from above 2 cells that text cleaning for train,validation and test set takes 8+ minutes that means we are losing some of our vital times for training  on tpu which is 3 hours(max). so it would be a good idea if we create another kernel and save above 2 cells newly updated train,val and test_data as kernels output then using those files we can quickly import our new train,test and validation data here,which will save time for training model on TPU **

**Roc-Auc Evaluation metric**

In [ ]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

# Tokenize(encode) comments

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

**Load bert tokenizer**

In [ ]:
MODEL = 'jplu/tf-xlm-roberta-large'
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

save_path = '/kaggle/working/xlmr_large/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
tokenizer.save_pretrained(save_path)



**Encode comments**

In [ ]:
# %%time
# x_train = regular_encode(train.comment_text.astype(str), 
#                       tokenizer, maxlen=maxlen)
# x_valid = regular_encode(val.comment_text.astype(str).values, 
#                       tokenizer, maxlen=maxlen)
# x_test = regular_encode(test_data.content.astype(str).values, 
#                      tokenizer, maxlen=maxlen)

# y_valid = val.toxic.values
# y_train = train.toxic.values

In [ ]:
# np.save('/kaggle/working/xtrain.npy', x_train)
# np.save('/kaggle/working/ytrain.npy', y_train)
# np.save('/kaggle/working/xtest.npy', x_test)
# np.save('/kaggle/working/xval.npy', x_valid)
# np.save('/kaggle/working/yval.npy', y_valid)

In [ ]:
x_train = np.load('/kaggle/input/kernel9/drive-download-20200522T103552Z-001/xtrain_9.npy')
x_test = np.load('/kaggle/input/kernel9/drive-download-20200522T103552Z-001/xtest_9.npy')
x_valid = np.load('/kaggle/input/kernel9/drive-download-20200522T103552Z-001/xval_9.npy')
y_train = np.load('/kaggle/input/kernel9/drive-download-20200522T103552Z-001/ytrain_9.npy')
y_valid = np.load('/kaggle/input/kernel9/drive-download-20200522T103552Z-001/yval_9.npy')


**previously we lost 8+ minutes and here ~12 minutes,sum them and we have lost 20+ minutes, which is almost 1 epoch training time here..!! :(**

In [ ]:
MODEL

**Train. validation and testing dataset**

In [ ]:
%%time
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [ ]:
print(strategy.num_replicas_in_sync)

# Focal Loss

In [ ]:
from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=.2):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

# Build the model and check summary

In [ ]:
def build_model(transformer, loss='binary_crossentropy', max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    x = tf.keras.layers.Dropout(0.3)(cls_token)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=3e-5), loss=loss, metrics=[tf.keras.metrics.AUC()])
    
    return model

In [ ]:
%%time
with strategy.scope():
    transformer_layer = transformers.TFXLMRobertaModel.from_pretrained(MODEL)
    model = build_model(transformer_layer,loss='binary_crossentropy', max_len=maxlen)
model.summary()

**Define Define ReduceLROnPlateau callback**

In [ ]:
def callback():
    cb = []

    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',  
                                    factor=0.3, patience=2, 
                                    verbose=1, mode='auto', 
                                    epsilon=0.0001, cooldown=1, min_lr=0.000001)
    cb.append(reduceLROnPlat)
    log = CSVLogger('log.csv')
    cb.append(log)

    RocAuc = RocAucEvaluation(validation_data=(x_valid, y_valid), interval=1)
    cb.append(RocAuc)
    
    return cb

# Visualize model architecture

In [ ]:
SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

# Learning rate schedule

In [ ]:
def build_lrfn(lr_start=0.000001, lr_max=0.000002, 
               lr_min=0.0000001, lr_rampup_epochs=7, 
               lr_sustain_epochs=0, lr_exp_decay=.87):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 7))

lrfn = build_lrfn()
plt.plot([i for i in range(35)], [lrfn(i) for i in range(35)]);

In [ ]:
model_path = 'jigsawMultilingual.hdf5'
model_path1 = '/kaggle/working/jigsawMultilingual.hdf5'

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', mode='max', save_best_only=True)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=2, 
                   restore_best_weights=True, verbose=1)
lr_callback = LearningRateScheduler(lrfn, verbose=1)

callback_list = [checkpoint,  lr_callback]

# Training

In [ ]:
%%time
N_STEPS = x_train.shape[0] // BATCH_SIZE
EPOCHS = 2
train_history = model.fit(
    train_dataset,
    steps_per_epoch=N_STEPS,
    validation_data=valid_dataset,
    callbacks=callback_list,
    epochs=EPOCHS
)

In [ ]:
v=valid.copy()[['id','toxic']]
v['toxic'] = model.predict(valid_dataset, verbose=1)
v.to_csv('understanding_cross_lingual_models_submission_val.csv', index=False)
print(roc_auc_score(valid['toxic'], v['toxic']))
v

In [ ]:
if os.path.exists(model_path1):
    model.load_weights(model_path1)

we train it for 2 more epochs on the validation set, which is significantly smaller but contains a mixture of different languages.

In [ ]:
%%time
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    callbacks=callback_list,
    epochs= EPOCHS
)

In [ ]:
if os.path.exists(model_path1):
    model.load_weights(model_path1)

In [ ]:
log_dir = "/kaggle/working/log.csv"
if os.path.exists(log_dir):
    os.remove(log_dir)

# Make Submission

In [ ]:
sub = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/' + 'sample_submission.csv')
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('understanding_cross_lingual_models_submission_test.csv', index=False)

In [ ]:
sub

**References**
* [Jigsaw TPU: XLM-Roberta](https://www.kaggle.com/xhlulu/jigsaw-tpu-xlm-roberta)
* [Jigsaw TPU: DistilBERT with Huggingface and Keras](https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras)
* [Jigsaw TPU: BERT with Huggingface and Keras](https://www.kaggle.com/miklgr500/jigsaw-tpu-bert-with-huggingface-and-keras)
* [8 Excellent Pretrained Models to get you Started with Natural Language Processing (NLP)](https://www.analyticsvidhya.com/blog/2019/03/pretrained-models-get-started-nlp/)
* [Introduction to PyTorch-Transformers: An Incredible Library for State-of-the-Art NLP (with Python code)](https://www.analyticsvidhya.com/blog/2019/07/pytorch-transformers-nlp-python/)

* [facebookresearch/XLM](https://github.com/facebookresearch/XLM)

* [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)

* [Exploring Distributional Representations and Machine Translation for Aspect-based Cross-lingual Sentiment Classification](https://www.researchgate.net/publication/309312650_Exploring_Distributional_Representations_and_Machine_Translation_for_Aspect-based_Cross-lingual_Sentiment_Classification)
* [Fastai with 🤗 Transformers (BERT, RoBERTa, ...)](https://www.kaggle.com/maroberti/fastai-with-transformers-bert-roberta)
* [Jigsaw Multilingual Toxicity : EDA + Models](https://www.kaggle.com/tarunpaparaju/jigsaw-multilingual-toxicity-eda-models)
* [Flower Classification with TPUs - EDA and Baseline](https://www.kaggle.com/dimitreoliveira/flower-classification-with-tpus-eda-and-baseline/notebook)